In [ ]:
import numpy as np
import odc.geo.xr  # noqa: F401
from datacube_compute import geomedian_with_mads
from distributed import Client
from odc.stac import configure_rio

from datacube import Datacube

In [ ]:
configure_rio(cloud_defaults=True, aws=dict(aws_unsigned=True))

In [ ]:
client = Client()
client

In [ ]:
# Study site in South East Tasmania
bbox = [146.2357, -43.6796, 147.147, -42.9305]
resolution = 20

# All of Tasmania
# bbox = [144.0, -43.6, 148.0, -40.0]
# resolution = 100

lon = (bbox[0], bbox[2])
lat = (bbox[1], bbox[3])

year = "2017"
product = "sentinel1_grd_gamma0_20m"

dc = Datacube()

In [ ]:
data = dc.load(
    product=product,
    lon=lon,
    lat=lat,
    time=year,
    dask_chunks=dict(x=2024, y=2024),
    output_crs="epsg:3577",
    resolution=(-resolution, resolution),
    group_by="solar_day",
)

data = data.where(data.vv != 0)
data.attrs["nodata"] = np.nan

for dv in data.data_vars.values():
    data[dv].attrs.pop("nodata")

data

In [ ]:
geomad = geomedian_with_mads(data)

for band in ["vv", "vh"]:
    geomad[f"{band}_mean"] = data[band].mean("time")
    geomad[f"{band}_std"] = data[band].std("time")

    # Rename bands
    geomad = geomad.rename({band: f"{band}_gm"})

geomad

In [ ]:
computed = geomad.compute()

In [ ]:
# Band vv_gm and vh_gm are the geomedian values
computed["vv_gm"].odc.explore(robust=True)

In [ ]:
# We can write out to file using the below
BAND = "gm_vv"
# computed[BAND].odc.write_cog(f"s1_mosaic_{year}_{BAND}.tif")